In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

print(tf.__version__)

In [ ]:
# loading dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
input_shape = (28, 28, 1)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train = x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test = x_test / 255.0

In [ ]:
# one hot encoding
# y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [ ]:
record_defaults = [tf.constant([], dtype=tf.float32)] * 10

y_train = tf.data.experimental.CsvDataset(
    filenames=["noisy_y_train.csv"], record_defaults=record_defaults
)

In [ ]:
# Convert the CsvDataset to a list of EagerTensors
y_train = [tf.convert_to_tensor(value) for value in y_train]

In [ ]:
batch_size = 64
num_classes = 10
num_epochs = 30

In [ ]:
optimizer = tf.keras.optimizers.SGD(name="SGD")
cce = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(
            32,
            (5, 5),
            padding="same",
            activation="relu",
            input_shape=input_shape,
            name="conv1_1",
        ),
        tf.keras.layers.Conv2D(
            32, (5, 5), padding="same", activation="relu", name="conv1_2"
        ),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(
            64, (3, 3), padding="same", activation="relu", name="conv2_1"
        ),
        tf.keras.layers.Conv2D(
            64, (3, 3), padding="same", activation="relu", name="conv2_2"
        ),
        tf.keras.layers.MaxPool2D(strides=(2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu", name="FC_1"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation="relu", name="FC_2"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax", name="FC_softmax"),
    ]
)

model.compile(
    optimizer=tf.keras.optimizers.SGD(name="SGD"),
    loss="categorical_crossentropy",
    metrics=["acc"],
)

In [ ]:
# Load gradients from the checkpoint file
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint.restore("epoch_layer_gradients.ckpt-1").expect_partial()

In [ ]:
# get indices with noisy data
indices = (np.loadtxt("noisy_indices.csv", delimiter=",")).astype(int)
indices = indices.tolist()

# convert eagertensor to numpy
y_train_numpy = [value.numpy() for value in y_train]

# create a sample of 1000 values using the 6k noisy examples
noisy_values_y = [y_train_numpy[i] for i in indices]
noisy_sample_y = noisy_values_y[:6000:6]

noisy_values_x = [x_train[i] for i in indices]
noisy_sample_x = noisy_values_x[:6000:6]

In [ ]:
# get clean indices
total_indices = list(range(60000))
clean_indices = [value for value in total_indices if value not in indices]

# create a sample of 1000 values using the remaining 54k clean examples
clean_values_y = [y_train_numpy[i] for i in clean_indices]
clean_sample_y = clean_values_y[:54000:54]

clean_values_x = [x_train[i] for i in clean_indices]
clean_sample_x = clean_values_x[:54000:54]

In [ ]:
print(noisy_values_x[0].shape)
noisy_sample = zip(noisy_sample_x, noisy_sample_y)
clean_sample = zip(clean_sample_x, clean_sample_y)

In [ ]:
noisy_xTensors = tf.convert_to_tensor(noisy_sample_x)
clean_xTensors = tf.convert_to_tensor(clean_sample_x)

In [ ]:
# change clean to noisy if needed
original_pred = model(clean_xTensors)

In [ ]:
def avg_noisy_pred(sample):
    outputshape = (1000, 10, 1)
    output = tf.zeros(outputshape, dtype=tf.float32)
    # add noise to input a make a prediction
    for i in range(0, 5):
        noise = np.random.normal(size=tf.shape(sample))
        noisy_images = sample + noise

        # pass to output for averaging
        output = output + model(noisy_images)

    return output / 5

In [ ]:
avg_noisy_output = avg_noisy_pred(clean_xTensors)

In [ ]:
from keras.metrics import CategoricalAccuracy

num_epochs = 1
accuracy_metric = CategoricalAccuracy()

epoch_gradients = []
# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    for batch_start in range(0, len(x_train), batch_size):
        # Extract a batch of data
        # X_batch = x_train[batch_start:batch_start + batch_size]
        # y_batch = y_train[batch_start:batch_start + batch_size]

        # Forward pass and compute gradients
        with tf.GradientTape() as tape:
            # predictions = model(X_batch)

            predictions = avg_noisy_output

            loss = cce(y_batch, predictions)
            loss_value = tf.reduce_mean(loss)

        # Backward pass and update weights
        gradients = tape.gradient(loss_value, model.trainable_variables)
        # optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        # Get accuracy of the epoch
        accuracy_metric.update_state(y_batch, predictions)

    accuracy = accuracy_metric.result()
    accuracy_metric.reset_states()  # reset for next epoch

    epoch_gradients.append(gradients)

    print(f"  Loss: {loss_value.numpy()}, Acc: {accuracy.numpy()}")